# VGG Network

### Import needed libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Add, ReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.optimizers import SGD


import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

### GPU Config

In [ ]:
np.random.seed(7)

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(sess)

num_classes = None

### Define loading CIFAR-10 function

In [ ]:
def load_data(num_images=None):
    global num_classes
    # Cargar el conjunto de datos CIFAR-10
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()

    # Seleccionar un número específico de imágenes si se proporciona num_images
    if num_images is not None:
        x_train = x_train[:num_images]
        y_train = y_train[:num_images]

    # Normalizar los valores de píxeles a un rango entre 0 y 1
    x_train = x_train.astype('float32') / 255.0
    x_test = x_test.astype('float32') / 255.0

    # One-hot encode de las etiquetas
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    num_classes = y_test.shape[1]

    return (x_train, y_train), (x_test, y_test)

### Define training model function

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving weights1.zip to weights1.zip


In [ ]:
import zipfile
import os
zip_file_path = './weights1.zip'

# Open the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Extract all the contents into the same directory
    zip_ref.extractall(os.path.dirname(zip_file_path))

In [ ]:
ls

accuracy_resnet_5000_0.001.png  weights1.zip                    weights_resnet_5000_0.001.h5
loss_resnet_5000_0.001.png      weights_resnet_25000_0.0001.h5
sample_data/                    weights_resnet_25000_0.001.h5


In [ ]:
weights_path = './weights_resnet_25000_0.0001.h5'
weights_file = Path(weights_path)

def create_model():
    global num_classes
    dropout=0.5

    input_image = Input(shape=(32,32,3))
    x1 = Conv2D(64, (3, 3),padding='same', activation='relu')(input_image)
    x1 = Conv2D(64, (3, 3),padding='same', activation='relu')(x1)
    x1 = MaxPooling2D((2, 2))(x1)

    res = Conv2D(128, (3, 3),padding='same', activation='relu')(x1)
    x1 = Conv2D(128, (3, 3),padding='same')(x1)
    x1 = Add()([res, x1])
    x1 = ReLU()(x1)
    x1 = MaxPooling2D((2, 2))(x1)

    res = Conv2D(256, (3, 3),padding='same', activation='relu')(x1)
    x1 = Conv2D(256, (3, 3),padding='same', activation='relu')(x1)
    x1 = Conv2D(256, (1, 1),padding='same', activation='relu')(x1)
    x1 = Add()([res, x1])
    x1 = ReLU()(x1)
    x1 = MaxPooling2D((2, 2))(x1)

    res = Conv2D(512, (3, 3),padding='same', activation='relu')(x1)
    x1 = Conv2D(512, (3, 3),padding='same', activation='relu')(x1)
    x1 = Conv2D(512, (1, 1),padding='same', activation='relu')(x1)
    x1 = Add()([res, x1])
    x1 = ReLU()(x1)
    x1 = MaxPooling2D((2, 2))(x1)

    res = Conv2D(512, (3, 3),padding='same', activation='relu')(x1)
    x1 = Conv2D(512, (3, 3),padding='same', activation='relu')(x1)
    x1 = Conv2D(512, (1, 1),padding='same', activation='relu')(x1)
    x1 = Add()([res, x1])
    x1 = ReLU()(x1)
    x1 = MaxPooling2D((2, 2))(x1)

    x1 = Flatten()(x1)

    x=Dense(4096, activation='relu', kernel_constraint=max_norm(3))(x1)
    x=Dropout(dropout)(x)
    x=Dense(4096, activation='relu', kernel_constraint=max_norm(3))(x)
    x=Dropout(dropout)(x)

    out= Dense(num_classes, activation='softmax')(x)

    model = Model(inputs = input_image, outputs = out);

    #Load parameters
    if weights_file.is_file():
      model.load_weights(weights_path)
    return model

def train_model(x_train, y_train, x_test, y_test, learning_rate, num_img_train):

  optimizer = SGD(learning_rate=learning_rate, momentum=0.9, nesterov=False)
  model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

  # Entrenar el modelo
  history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=20, batch_size=64)

  # Mostrar o guardar gráficos, guardar modelos, etc.
  plot_and_save_results(history, num_img_train, learning_rate)

### Define ploting function

In [ ]:
def plot_and_save_results(history, num_img_train, learning_rate):
    # Accuracy figure
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Accuracy for '+str(num_img_train)+' images with learning rate '+str(learning_rate))
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')

    accuracy_img_name = "accuracy_resnet_"+ str(num_img_train) +"_"+ str(learning_rate) +".png"
    print(accuracy_img_name)
    plt.savefig(accuracy_img_name)
    # plt.show()
    plt.close()


    # Loss figure
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Loss for '+str(num_img_train)+' images with learning rate '+str(learning_rate))
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')

    loss_img_name = 'loss_resnet_'+str(num_img_train)+'_'+str(learning_rate)+'.png'
    plt.savefig(loss_img_name)
    # plt.show()
    plt.close()


    # Save weights
    weights_path = './weights_resnet_'+str(num_img_train) +"_"+ str(learning_rate) +'.h5'
    weights_file = Path(weights_path)
    model.save_weights(weights_path)

### Fit the model

In [ ]:
num_images_list = 25000
learning_rates = 0.0001

(x_train, y_train), (x_test, y_test) = load_data(num_images_list)
print(f"Training with {num_images_list} images and learning rate {learning_rates}")
model = create_model()
train_model(x_train, y_train, x_test, y_test, learning_rates, num_images_list)

Training with 25000 images and learning rate 0.0001
Epoch 1/20
391/391 [==============================] - 17s 36ms/step - loss: 2.1454 - accuracy: 0.2063 - val_loss: 2.1059 - val_accuracy: 0.2505
Epoch 2/20
391/391 [==============================] - 14s 35ms/step - loss: 2.1132 - accuracy: 0.2200 - val_loss: 2.0659 - val_accuracy: 0.2658
Epoch 3/20
391/391 [==============================] - 14s 36ms/step - loss: 2.0803 - accuracy: 0.2288 - val_loss: 2.0323 - val_accuracy: 0.2703
Epoch 4/20
391/391 [==============================] - 14s 36ms/step - loss: 2.0579 - accuracy: 0.2294 - val_loss: 2.0114 - val_accuracy: 0.2721
Epoch 5/20
391/391 [==============================] - 14s 35ms/step - loss: 2.0394 - accuracy: 0.2389 - val_loss: 1.9951 - val_accuracy: 0.2763
Epoch 6/20
391/391 [==============================] - 14s 35ms/step - loss: 2.0226 - accuracy: 0.2438 - val_loss: 1.9834 - val_accuracy: 0.2752
Epoch 7/20
391/391 [==============================] - 14s 35ms/step - loss: 2.0108 -

In [ ]:
!zip -r /content/results_resnet.zip /content/

  adding: content/ (stored 0%)
  adding: content/.config/ (stored 0%)
  adding: content/.config/default_configs.db (deflated 98%)
  adding: content/.config/.last_survey_prompt.yaml (stored 0%)
  adding: content/.config/.last_update_check.json (deflated 22%)
  adding: content/.config/logs/ (stored 0%)
  adding: content/.config/logs/2023.12.19/ (stored 0%)
  adding: content/.config/logs/2023.12.19/14.20.16.265569.log (deflated 86%)
  adding: content/.config/logs/2023.12.19/14.20.35.154355.log (deflated 57%)
  adding: content/.config/logs/2023.12.19/14.19.39.750127.log (deflated 91%)
  adding: content/.config/logs/2023.12.19/14.20.35.996145.log (deflated 56%)
  adding: content/.config/logs/2023.12.19/14.20.25.266295.log (deflated 58%)
  adding: content/.config/logs/2023.12.19/14.20.06.327238.log (deflated 58%)
  adding: content/.config/gce (stored 0%)
  adding: content/.config/.last_opt_in_prompt.yaml (stored 0%)
  adding: content/.config/configurations/ (stored 0%)
  adding: content/.con